In [1]:
import os, sys
dir1 = os.path.abspath('..\..')
sys.path.append(dir1+ "\\")

import MetaTrader5 as mt
from datetime import datetime as dt

# custom
from my_jupyter.user import User
from my_jupyter.strategy_manager import StrategyManager
from my_jupyter.operation import Operation
from my_jupyter.strategies.min_max_strategy import MinMaxStrategy
from my_jupyter.market_data_repository import MarketDataRepository
# from Mbox import Mbox
# from boleta_1 import gerar_boleta
# from gerenciamento import Gerenciamento

## GUI's
from daemon_tools import DaemonThreadMini


In [2]:
from my_jupyter.filters.moving_average import MovingAverageFilter
from my_jupyter.filters.consequent_limits_counter import ConsequentLimitsCounterFilter
from my_jupyter.filters.directioned_bars_filter import DirectionedBarsFilter


filters = [DirectionedBarsFilter(candles=5), MovingAverageFilter()]
strategies = [MinMaxStrategy(filters=filters)]
operations = [Operation("WINV23", 1, mt.TIMEFRAME_M1, can_buy=True, can_sell=True)]
user = User(strategies=strategies, operations=operations, trading_time_start=dt.now())
market_data_repository = MarketDataRepository()
strategy = StrategyManager(market_data_repository, user)

# PARAMETROS MAIN
kwargs = {
    "strategy": strategy,
}


def main(**kwargs):
    _shift = 0
    strategy: StrategyManager = kwargs["strategy"]
    strategy.run_strategies()
    # print(f"{dt.now().strftime('%d/%m/%Y, %H:%M:%S')} >>> D-{_shift}")
def exec_daemon(**kwargs):
    main(**kwargs)
    # alarme.ring_alarme()   

In [3]:
# gerenciamento = Gerenciamento(mt, timeframe, timeframes_dict)

try:
    main(**kwargs)
except Exception as e:
    print(e.with_traceback(__tb=None))
    # Mbox.Alerta("ERRO", "busca de sinais com erro")


counting: 4
counting: 4


In [4]:
from my_jupyter.daemons.daemon_tools import DaemonThreadMini


thread_exec = DaemonThreadMini(exec_daemon, seconds=0, mseconds=100, **kwargs)
thread_exec.start()

counting: 4
counting: 4


In [ ]:
# bars = mt.mt.copy_rates_from_pos("PETR4", mt.mt.TIMEFRAME_D1, 0, 1000000-1)[::-1]


counting: -5
counting: -5
counting: -5
counting: -5


counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5
counting: -5